# 06 — Açıklanabilirlik (Explainability)

Rezervasyon iptal tahmin modelinin neden bu kararı verdiğini açıklar.

Bu notebook:
- **Permutation Importance** — global özellik önemleri (model-agnostik)
- **SHAP Summary Plot** — özellik katkı dağılımı
- **SHAP Waterfall** — bireysel tahmin açıklaması
- **SHAP Dependence Plot** — özellik etkileşimleri

> **Not:** SHAP ağaç tabanlı modeller için çok hızlıdır (TreeExplainer). LR/SVM için KernelExplainer kullanılır (yavaş).

In [ ]:
import sys

sys.path.insert(0, "..")

import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

## 1. Model & Veri

In [ ]:
from src.config import Paths
from src.io import load_latest_model, read_input_dataset
from src.preprocess import preprocess_basic
from src.split import split_dataset

paths = Paths()
model = load_latest_model(paths)

df, _ = read_input_dataset(paths.raw_data)
df = preprocess_basic(
    df, target_col="is_canceled", label_map={"no": 0, "yes": 1, 0: 0, 1: 1}
)
split = split_dataset(df, target_col="is_canceled")
X_test, y_test = split.X_test, split.y_test

# Küçük bir örnek kullan (hız için)
N_SAMPLE = min(2000, len(X_test))
X_sample = X_test.sample(N_SAMPLE, random_state=42)
y_sample = y_test.loc[X_sample.index]
print(f"Örnek boyutu: {N_SAMPLE}")

## 2. Permutation Importance (Global, Model-Agnostik)

In [ ]:
from src.explain import compute_permutation_importance

perm_imp = compute_permutation_importance(
    model, X_sample, y_sample.values, n_repeats=5, random_state=42
)

# İlk 20 özellik
top20 = perm_imp.head(20).copy()

fig, ax = plt.subplots(figsize=(10, 6))
colors = ["#ef4444" if v > 0 else "#94a3b8" for v in top20["importance_mean"]]
ax.barh(
    top20["feature"],
    top20["importance_mean"],
    xerr=top20["importance_std"],
    color=colors,
    edgecolor="none",
    capsize=3,
)
ax.set_title("Permutation Importance (Top 20)")
ax.set_xlabel("Ortalama AUC Düşüşü")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

top20

## 3. SHAP Analizi

In [ ]:
import shap
from src.explain import compute_shap_values

shap_result = compute_shap_values(model, X_sample)
shap_values = shap_result.get("shap_values")
feature_names = shap_result.get("feature_names", X_sample.columns.tolist())

if shap_values is not None:
    print(f"SHAP değerleri hesaplandı. Shape: {np.asarray(shap_values).shape}")
else:
    print("SHAP hesaplanamadı (KernelExplainer için X_sample boyutunu küçültün).")

In [ ]:
if shap_values is not None:
    # Summary plot — beeswarm
    shap.summary_plot(
        shap_values,
        X_sample,
        feature_names=feature_names,
        show=True,
        max_display=20,
    )
    plt.suptitle("SHAP Summary Plot (Beeswarm)", y=1.01)
    plt.tight_layout()
    plt.show()

### 3.2 SHAP Waterfall — Tek Tahmin Açıklaması

In [ ]:
if shap_values is not None:
    # En yüksek iptal olasılıklı kaydı seç
    y_prob = model.predict_proba(X_sample)[:, 1]
    idx_max = np.argmax(y_prob)

    sv_arr = np.asarray(shap_values)
    # Waterfall plot için shap.Explanation oluştur
    try:
        explainer_obj = shap_result.get("explainer")
        exp = shap.Explanation(
            values=sv_arr[idx_max],
            base_values=explainer_obj.expected_value
            if hasattr(explainer_obj, "expected_value")
            else 0.0,
            data=X_sample.iloc[idx_max].values,
            feature_names=feature_names,
        )
        shap.waterfall_plot(exp, max_display=15, show=True)
    except Exception as e:
        print(f"Waterfall plot oluşturulamadı: {e}")

### 3.3 SHAP Dependence Plot — En Önemli Özellik

In [ ]:
if shap_values is not None:
    sv_arr = np.asarray(shap_values)
    mean_abs_shap = np.abs(sv_arr).mean(axis=0)
    top_feature_idx = np.argmax(mean_abs_shap)
    top_feature = feature_names[top_feature_idx]
    print(f"En önemli SHAP özelliği: {top_feature}")

    fig, ax = plt.subplots()
    shap.dependence_plot(
        top_feature_idx,
        sv_arr,
        X_sample,
        feature_names=feature_names,
        ax=ax,
        show=False,
    )
    ax.set_title(f"SHAP Dependence: {top_feature}")
    plt.tight_layout()
    plt.show()

## 4. Rapor Kaydetme

In [ ]:
from src.explain import save_explain_report

report = {
    "permutation_importance": perm_imp.to_dict(orient="records"),
    "shap_computed": shap_values is not None,
    "n_sample": N_SAMPLE,
}

save_explain_report(report, paths)
print("Açıklanabilirlik raporu kaydedildi:", paths.reports_dir / "explain_report.json")

## Sonuç

| Teknik | Kapsam | Hız |
|--------|--------|-----|
| Permutation Importance | Global | Hızlı |
| SHAP TreeExplainer | Global + Lokal | Çok Hızlı (ağaç) |
| SHAP KernelExplainer | Global + Lokal | Yavaş (genel) |
| SHAP Waterfall | Lokal (1 kayıt) | Anlık |

Resmi açıklanabilirlik raporu: `python main.py explain`